### gRPC Server

In [41]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [42]:
from tensorflow.core.framework import tensor_pb2, tensor_shape_pb2, types_pb2

def dtypes_as_dtype(dtype):
    if dtype == "float32":
        return types_pb2.DT_FLOAT
    raise Exception("dtype %s is not supported" % dtype)


def make_tensor_proto(data):
    shape = data.shape
    dims = [tensor_shape_pb2.TensorShapeProto.Dim(size=i) for i in shape]
    proto_shape = tensor_shape_pb2.TensorShapeProto(dim=dims)

    proto_dtype = dtypes_as_dtype(data.dtype)

    tensor_proto = tensor_pb2.TensorProto(dtype=proto_dtype, tensor_shape=proto_shape)
    tensor_proto.tensor_content = data.tostring()

    return tensor_proto

In [43]:
def np_to_protobuf(data):
    return make_tensor_proto(data)
    # return tf.make_tensor_proto(data, shape=data.shape)

In [1]:
from PIL import Image
from io import BytesIO
import numpy as np
import requests

def load_image(path, from_url=True, process=True):
    """
    Custom preprocessing function. 
    """
    if from_url:
        response = requests.get(path)
        img = Image.open(BytesIO(response.content))
    else:
        img = Image.open(path)
    if process:
        img = img.resize((224,224), Image.Resampling.LANCZOS)
        img = np.array(img, dtype=np.float32)
        img = img * (1./255)
    return np.asarray([img])

In [2]:
url = "https://t4.ftcdn.net/jpg/00/97/58/97/360_F_97589769_t45CqXyzjz0KXwoBZT9PRaWGHRk5hQqQ.jpg"

In [3]:
X = load_image(url)

In [ ]:
docker run -it --rm \
  -p 8500:8500 \
  zoomcamp-10-model:xception-v4-001

In [7]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [9]:
pb_request = predict_pb2.PredictRequest()

print(type(pb_request.inputs))

pb_request.model_spec.name = 'cats_and_dogs'
pb_request.model_spec.signature_name = 'serving_default'

print(type(pb_request.inputs))

<class 'google.protobuf.pyext._message.MessageMapContainer'>
<class 'google.protobuf.pyext._message.MessageMapContainer'>


In [10]:
pb_request.inputs['input_1'].CopyFrom(np_to_protobuf(X))

C:\Users\dell\AppData\Local\Temp\ipykernel_16012\3268910938.py:17: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = data.tostring()


In [11]:
print(type(pb_request.inputs))

<class 'google.protobuf.pyext._message.MessageMapContainer'>


In [12]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8500: Connection refused"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8500: Connection refused {created_time:"2023-12-27T19:35:42.5937213+00:00", grpc_status:14}"
>

In [ ]:
preds = pb_response.outputs['dense_3'].float_val

In [25]:
preds

[1.0, 6.131807301495984e-12]

In [29]:
!python gateway.py

*********************Host:  localhost:8500
{'cat': 1.0, 'dog': 6.131807301495984e-12}


### Rest API Server

In [61]:
# X.tolist()
import requests
import json

In [58]:
model_url = "http://localhost:8501/v1/models/cats_and_dogs:predict"

In [49]:
X[0].shape

(224, 224, 3)

In [62]:
response = requests.post(model_url, data=json.dumps({"instances": X.tolist()}))

In [66]:
response.json()['predictions'][0]

[1.0, 6.1318073e-12]

In [57]:
!curl http://localhost:8501/v1/models/cats_and_dogs

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   154  100   154    0     0    835      0 --:--:-- --:--:-- --:--:--   836


In [68]:
!python gateway.py

{'cat': 1.0, 'dog': 6.1318073e-12}
